Imports & Constants

In [366]:
# run '!pip install <package name>' if module not found error shows up
from rsome import ro 
from rsome import grb_solver as grb
import pandas as pd
import numpy as np
import folium
import random
from folium.plugins import BeautifyIcon

LOWERED_SERVICE_COUNT = 2
LOWERED_SERVICE_COUNT2 = 10
CENTER = [1.3521, 103.8198] # singapore center coordinates

Get Data

In [354]:
# read and clean routes

routes_ori = pd.read_csv('bus_routes.csv')
routes_ori = routes_ori.drop(columns = ["WD_FirstBus" , "Operator" , "WD_LastBus", "SAT_FirstBus", "SAT_LastBus" , "SUN_FirstBus", "SUN_LastBus", "StopSequence"])
routes_ori = routes_ori.drop(columns = routes_ori.columns[0])
routes_ori = routes_ori.drop(routes_ori[routes_ori.ServiceNo == '700'].index) # remove defunt bus 700
routes_ori

,ServiceNo,Direction,BusStopCode,Distance
0,10,1,75009,0.0
1,10,1,76059,0.6
2,10,1,76069,1.1
3,10,1,96289,2.3
4,10,1,96109,2.7
...,...,...,...,...
26312,NR8,1,43839,23.1
26313,NR8,1,43649,23.4
26314,NR8,1,43589,23.9
26315,NR8,1,43579,24.3


In [355]:
stops_loc_ori = pd.read_csv('bus_stops.csv')
stops_loc_ori = stops_loc_ori.drop(columns = stops_loc_ori.columns[0])
stops_loc_ori = stops_loc_ori.drop(columns = ["RoadName" , "Description"])
stops_loc_ori.BusStopCode = stops_loc_ori.BusStopCode.apply(lambda x: str(x))
stops_loc_ori

,BusStopCode,Latitude,Longitude
0,481,1.383764,103.758300
1,1012,1.296848,103.852536
2,1013,1.297710,103.853225
3,1019,1.296990,103.853022
4,1029,1.296673,103.854414
...,...,...,...
5016,99139,1.388195,103.987234
5017,99161,1.390262,103.992957
5018,99171,1.391128,103.991021
5019,99181,1.387754,103.988503


Global functions

In [382]:
def get_stops_loc(stops_loc_ori, stops):
    stops_loc = []
    for i in range(len(stops)):
        stop = stops[i]
        while stop[0] == '0':
            stop = stop[1:]        
        lat = stops_loc_ori[stops_loc_ori.BusStopCode == stop]['Latitude'].values[0]
        lon = stops_loc_ori[stops_loc_ori.BusStopCode == stop]['Longitude'].values[0]
        stops_loc.append([lat, lon])
    return stops_loc

def display_map(stops, stops_loc):
    map_sg = folium.Map(location=CENTER, zoom_start=15)
    for i in range(len(stops_loc)):
        lat = stops_loc[i][0]
        lon = stops_loc[i][1]
        folium.Marker(
        location=[lat, lon],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=stops[i],
            spin=True,
            text_color='red',
            background_color="#FFF",
            border_width = 1,
            inner_icon_style="font-size:13px;padding-top:-5px;"
        )
    ).add_to(map_sg)
    #display map
    return map_sg

def random_color_generator():
    hexadecimal = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
    return hexadecimal

def display_solution(path, stops_loc, services, stops, stops_loc_ori):
    services_used = list(set(np.where(path == 1)[2]))
    stops_used = list(set(list(np.where(path == 1)[0]) + list(np.where(path == 1)[1])))
    stops_used_name = list(map(lambda x: stops[x], stops_used))
    stops_used_loc = get_stops_loc(stops_loc_ori, stops_used_name)
    subpaths = []
    for i in range(len(np.where(path == 1)[0])):
        subpaths.append([np.where(path == 1)[0][i], np.where(path == 1)[1][i], np.where(path == 1)[2][i]])
    color_alloc = {}
    for i in services_used:
        color_alloc[i] = random_color_generator()
    map_sg = folium.Map(location=CENTER, zoom_start=15)
    for i in subpaths:
        start_index = i[0]
        end_index = i[1]
        service_index = i[2]
        service = services[service_index]
        start_lat = stops_loc[start_index][0]
        start_lon = stops_loc[start_index][1]
        end_lat = stops_loc[end_index][0]
        end_lon = stops_loc[end_index][1]
        coordinates = [[start_lat, start_lon], [end_lat, end_lon]]
        color = color_alloc[service_index]
        # folium.Marker(
        #     location=[lat, lon],
        #     icon=BeautifyIcon(
        #         icon_shape='marker',
        #         number=int(i),
        #         spin=True,
        #         text_color='red',
        #         background_color="#FFF",
        #         border_width = 1,
        #         inner_icon_style="font-size:13px;padding-top:-5px;"
        #     )
        # ).add_to(map_sg)
        my_PolyLine=folium.PolyLine(locations=coordinates,weight=5,color=color,tooltip =service)
        map_sg.add_child(my_PolyLine)
    for i in range(len(stops_used_loc)):
        lat = stops_used_loc[i][0]
        lon = stops_used_loc[i][1]
        folium.Marker(
            location=[lat, lon],
            icon=BeautifyIcon(
                icon_shape='marker',
                number=stops[stops_used[i]],
                spin=True,
                text_color='red',
                background_color="#FFF",
                border_width = 1,
                inner_icon_style="font-size:13px;padding-top:-5px;"
            )
        ).add_to(map_sg)
    return map_sg

def display_solution2(path, stops_loc, services, stops, stops_loc_ori):
    path_used = np.where(path == 1)[0].tolist()
    services_used = []
    stops_used = []
    subpaths = []
    for i in path_used:
        start = int(shaped_data[i][0])
        stop = int(shaped_data[i][1])
        service = int(shaped_data[i][2])
        services_used.append(service)
        stops_used.append(start)
        stops_used.append(stop)
        subpaths.append([start, stop, service])
    stops_used = list(set(stops_used))
    stops_used_name = list(map(lambda x: stops[x], stops_used))
    stops_used_loc = get_stops_loc(stops_loc_ori, stops_used_name)
    color_alloc = {}
    for i in services_used:
        color_alloc[i] = random_color_generator()
    map_sg = folium.Map(location=CENTER, zoom_start=15)
    for i in subpaths:
        start_index = i[0]
        end_index = i[1]
        service_index = i[2]
        service = services[service_index]
        start_lat = stops_loc[start_index][0]
        start_lon = stops_loc[start_index][1]
        end_lat = stops_loc[end_index][0]
        end_lon = stops_loc[end_index][1]
        coordinates = [[start_lat, start_lon], [end_lat, end_lon]]
        color = color_alloc[service_index]
        # folium.Marker(
        #     location=[lat, lon],
        #     icon=BeautifyIcon(
        #         icon_shape='marker',
        #         number=int(i),
        #         spin=True,
        #         text_color='red',
        #         background_color="#FFF",
        #         border_width = 1,
        #         inner_icon_style="font-size:13px;padding-top:-5px;"
        #     )
        # ).add_to(map_sg)
        my_PolyLine=folium.PolyLine(locations=coordinates,weight=5,color=color,tooltip =service)
        map_sg.add_child(my_PolyLine)
    for i in range(len(stops_used_loc)):
        lat = stops_used_loc[i][0]
        lon = stops_used_loc[i][1]
        folium.Marker(
            location=[lat, lon],
            icon=BeautifyIcon(
                icon_shape='marker',
                number=stops[stops_used[i]],
                spin=True,
                text_color='red',
                background_color="#FFF",
                border_width = 1,
                inner_icon_style="font-size:13px;padding-top:-5px;"
            )
        ).add_to(map_sg)
    return map_sg

# Approach 1

Format Data

In [367]:
# get unique bus service numbers

services = routes_ori.ServiceNo.unique()[:LOWERED_SERVICE_COUNT] # get first 2 services
services

array(['10', '100'], dtype=object)

In [368]:
# get a smaller sample of routes based on the first 2 services
# the memory required to run the full set is 5600 x 5600 x 560 x 2 bytes = ~35 GB

routes = routes_ori.loc[routes_ori['ServiceNo'].isin(services)]
routes


,ServiceNo,Direction,BusStopCode,Distance
0,10,1,75009,0.0
1,10,1,76059,0.6
2,10,1,76069,1.1
3,10,1,96289,2.3
4,10,1,96109,2.7
...,...,...,...,...
252,100,2,61031,21.9
253,100,2,61041,22.2
254,100,2,62111,22.6
255,100,2,62121,22.8


In [369]:
# get unique bus stops & loc

stops = routes.BusStopCode.unique()
print('number of stops: ', len(stops))
stops_loc = get_stops_loc(stops_loc_ori, stops)

number of stops:  222


In [370]:
# get shaped data array

services_count = len(services)
stops_count = len(stops)
min_dist = 0

data = np.zeros((stops_count, stops_count, services_count),dtype='float32') # initialize data array
for i in range(len(routes) - 1):
    next_index = i + 1
    if routes.iloc[i]['ServiceNo'] != routes.iloc[next_index]['ServiceNo'] or routes.iloc[i]['Direction'] != routes.iloc[next_index]['Direction']:
        continue # skip if the next row is not the same service or direction
    else:
        start_index = stops.tolist().index(routes.iloc[i]['BusStopCode'])
        end_index = stops.tolist().index(routes.iloc[next_index]['BusStopCode'])
        service_index = services.tolist().index(routes.iloc[i]['ServiceNo'])
        distance = routes.iloc[next_index]['Distance'] - routes.iloc[i]['Distance']
        if distance < min_dist or min_dist == 0:
            min_dist = distance
        data[start_index][end_index][service_index] = distance


Formulate Model

In [376]:
def best_path1(start_index, end_index, weight1, weight2, data, stops, services, min_dist):

    m = ro.Model('best path 1')
    M = len(stops)
    S = len(services)
    a = m.dvar(data.shape, vtype='B') # choice of route
    n = m.dvar((S,M), vtype='B') # number of transfers

    m.min(weight1 * sum(a[(i,j,k)] * data[(i,j,k)] for i in range(M) for j in range(M) for k in range(S)) + weight2 * (n.sum()))

    m.st(a[i,j,k] <= data[i,j,k] * 1/min_dist for i in range(M) for j in range(M) for k in range(S)) # if distance is 0, not allowed to choose
    m.st(sum(a[start_index, j, k] for j in range(M) for k in range(S)) == 1 ) # only one route from start to j
    m.st(sum(a[j, start_index, k] for j in range(M) for k in range(S)) == 0 )
    m.st(sum(a[i, end_index, k] for i in range(M) for k in range(S)) == 1 ) # only one route from i to end
    m.st(sum(a[end_index, i, k] for i in range(M) for k in range(S)) == 0 )
    for k in range(S):
        for l in range(M):
            if l != start_index and l != end_index:
                m.st(n[k,l] >= sum(a[l,i,k] for i in range(M) )- sum(a[i,l,k] for i in range(M) )) # calculate transfers
                m.st(n[k,l] >= sum(a[i,l,k] for i in range(M) ) - sum(a[l,i,k] for i in range(M) )) # calculate transfers
    m.st(n >= 0) # number of transfers cannot be negative
    for j in range(M): 
        if j != start_index and j != end_index:
            m.st(sum(a[i,j,k] for i in range(M) for k in range(S)) == sum(a[j,l,k] for l in range(M) for k in range(S))) # input = output

    m.solve(grb)

    return m.get(), a.get(), n.get()
    

## Execution

View Map

In [372]:
display_map(stops, stops_loc)

Set parameters and run solver

In [380]:
start = 15219
end = 80011
start_index = stops.tolist().index(str(start))
end_index = stops.tolist().index(str(end))
weight1 = 60/17 # 60 min / 17 km (average bus speed)
weight2 = 8 # average waiting time per bus
cost, path, transfer = best_path1(start_index, end_index, weight1, weight2, data, stops, services, min_dist)

# runtime: 6 min for 2 unique services and 222 stops

Being solved by Gurobi...
Solution status: 2
Running time: 0.1360s


View Solution on Map

In [383]:
print(cost)
display_solution(path, stops_loc, services, stops, stops_loc_ori)

62.23529472771815


# Approach 2

In [283]:
# get services

services = routes_ori.ServiceNo.unique()[:LOWERED_SERVICE_COUNT2] 
services

array(['10', '100'], dtype=object)

In [284]:
# get a smaller sample of routes for a smoother map view

routes = routes_ori.loc[routes_ori['ServiceNo'].isin(services)]
routes

,ServiceNo,Direction,BusStopCode,Distance
0,10,1,75009,0.0
1,10,1,76059,0.6
2,10,1,76069,1.1
3,10,1,96289,2.3
4,10,1,96109,2.7
...,...,...,...,...
252,100,2,61031,21.9
253,100,2,61041,22.2
254,100,2,62111,22.6
255,100,2,62121,22.8


In [285]:
services = routes.ServiceNo.unique()
stops = routes.BusStopCode.unique()
stops_loc = get_stops_loc(stops_loc_ori, stops)
service_indices = np.empty(len(services), dtype=list)
stop_indices = np.empty(len(stops), dtype=list)
shaped_data = None

def indexify_stops(stop):
    return stops.tolist().index(stop)

def indexify_services(service):
    return services.tolist().index(service)

def calc_dist_from_prev_stop(data):
    for i in range(len(data) - 1, 0, -1):
        if data.iloc[i]['Distance'] != 0 and data.iloc[i]['ServiceNo'] == data.iloc[i-1]['ServiceNo']:
            dist = data.iloc[i]['Distance'] - data.iloc[i-1]['Distance']
            if(dist < 0):
                dist = 0
            data.loc[i, 'Distance'] = round(dist, 1)
        else:
            data.loc[i, 'Distance'] = 0

def get_shaped_data(data):
    shaped_data_list = []
    for i in range(1, len(data)):
        if data.iloc[i]['Distance'] != 0 or data.iloc[i-1]['Direction'] == data.iloc[i]['Direction']:
            shaped_data_list.append([int(data.iloc[i - 1]['BusStopCode']), int(data.iloc[i]['BusStopCode']), int(data.iloc[i]['ServiceNo']), float(data.iloc[i]['Distance'])])
    return shaped_data_list

def get_indices_data(data):
    for i in range(len(data)):
        start_stop_idx = int(data[i][0])
        end_stop_idx = int(data[i][1])
        service_idx = int(data[i][2])
        if service_indices[service_idx] == None:
            service_indices[service_idx] = []
        if stop_indices[start_stop_idx] == None:
            stop_indices[start_stop_idx] = [[],[]]
        if stop_indices[end_stop_idx] == None:
            stop_indices[end_stop_idx] = [[],[]]
        service_indices[service_idx].append(i)
        stop_indices[start_stop_idx][0].append(i)
        stop_indices[end_stop_idx][1].append(i)
    

    

routes['ServiceNo'] = routes['ServiceNo'].apply(indexify_services)
routes['BusStopCode'] = routes['BusStopCode'].apply(indexify_stops)
calc_dist_from_prev_stop(routes)
print(routes)
shaped_data = np.array(get_shaped_data(routes))
shaped_data[np.isnan(shaped_data)] = 0 # replace nan with 0
# print(shaped_data)
get_indices_data(shaped_data)
# print(service_indices)
# print(stop_indices)


C:\Users\ZY\anaconda3\envs\gurobi37\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ZY\anaconda3\envs\gurobi37\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ZY\anaconda3\envs\gurobi37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

     ServiceNo  Direction  BusStopCode  Distance
0            0          1            0       0.0
1            0          1            1       0.6
2            0          1            2       0.5
3            0          1            3       1.2
4            0          1            4       0.4
..         ...        ...          ...       ...
252          1          2          218       0.5
253          1          2          219       0.3
254          1          2          220       0.4
255          1          2          221       0.2
256          1          2          146       0.5

[257 rows x 4 columns]


In [327]:
def best_path2(start_index, end_index, weight1, weight2, data, stop_indices, service_indices):
    dist_subset = []
    start_subset = []
    end_subset = []
    service_subset = []

    for i in range(len(data)):
        dist_subset.append(data[i][3])
        start_subset.append(data[i][0])
        end_subset.append(data[i][1])
        service_subset.append(data[i][2])

    dist_array = np.array(dist_subset)
    start_array = np.array(start_subset)
    end_array = np.array(end_subset)
    service_array = np.array(service_subset)

    data_len = len(data)
    M = len(stop_indices)
    N = len(service_indices)

    m = ro.Model('best path 2')
    a = m.dvar((data_len), vtype='B') # choice of routes
    s = m.dvar((N,M), vtype='B') # number of transfers

    m.min(weight1 * (a @ dist_array) + weight2 * s.sum())
    m.st(sum(a[i] for i in stop_indices[start_index][0]) == 1)
    m.st(sum(a[i] for i in stop_indices[end_index][1]) == 1)
    for i in range(N):
        for j in range(M):
            if j != start_index and j != end_index:
                service_rows = service_indices[i]
                start_rows = stop_indices[j][0]
                end_rows = stop_indices[j][1]
                start_serv_intersect = list(set(service_rows).intersection(set(start_rows)))
                end_serv_intersect = list(set(service_rows).intersection(set(end_rows)))
                if len(start_serv_intersect) == 0 and len(end_serv_intersect) == 0:
                    m.st(s[i,j] == 0)
                elif len(start_serv_intersect) == 1 and len(end_serv_intersect) == 0:
                    m.st(s[i,j] >= a[start_serv_intersect[0]])
                elif len(start_serv_intersect) == 0 and len(end_serv_intersect) == 1:
                    m.st(s[i,j] >= a[end_serv_intersect[0]])
                elif len(start_serv_intersect) == 1 and len(end_serv_intersect) == 1:
                    m.st(s[i,j] >= a[start_serv_intersect[0]] - a[end_serv_intersect[0]])   
                    m.st(s[i,j] >= a[end_serv_intersect[0]] - a[start_serv_intersect[0]])   
    for i in range(M):
        if i != start_index and i != end_index:
            m.st(sum(a[j] for j in stop_indices[i][0]) == sum(a[j] for j in stop_indices[i][1]))
    m.st(a >= 0, s >= 0)

    m.solve(grb)

    return m.get(), a.get(), s.get()
    

In [330]:
display_map(stops, stops_loc)

In [338]:
start = 16069
end = 18019
start_index = stops.tolist().index(str(start))
end_index = stops.tolist().index(str(end))
weight1 = round(60/17, 1) # 60 min / 17 km (average bus speed)
weight2 = 8 # average waiting time per bus (for transfers)
cost, path, transfers = best_path2(start_index, end_index, weight1, weight2, shaped_data, stop_indices, service_indices)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0040s


In [339]:
print(cost)
display_solution2(path, stops_loc, services, stops, stops_loc_ori)

191.0000000000001


In [292]:
service_rows = [1,2,3,4,5]
start_rows = [5,6,7,8,9]
list(set(service_rows).intersection(set(start_rows)))

[5]